In [26]:
%load_ext autoreload
%autoreload 2

import os
import glob
import shutil
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import autoeis as ae
from helpers import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Jones (2022) dataset

In [28]:
# Load EIS dataset
path_dataset = 'dataset/PJ131'
# path_eis_data = os.path.join(path_dataset, 'eis-raw')
path_eis_data = "dataset/JonesLee2022/raw-data/fixed-discharge/PJ131"

file_list = glob.glob(os.path.join(path_eis_data, '*.txt'))
file_list = [file for file in file_list if is_valid_eis_file(file)]
print(f"Number of files: {len(file_list)}")

file_list.sort(key=get_cycle_number)
file_list[:5]


Number of files: 256


['dataset/JonesLee2022/raw-data/fixed-discharge/PJ131/PJ131_002_01_GEIS_CA3.txt',
 'dataset/JonesLee2022/raw-data/fixed-discharge/PJ131/PJ131_002_03_GEIS_CA3.txt',
 'dataset/JonesLee2022/raw-data/fixed-discharge/PJ131/PJ131_002_07_GEIS_CA3.txt',
 'dataset/JonesLee2022/raw-data/fixed-discharge/PJ131/PJ131_002_05_GEIS_CA3.txt',
 'dataset/JonesLee2022/raw-data/fixed-discharge/PJ131/PJ131_002_09_GEIS_CA3.txt']

In [29]:
# Define the path to the folder where the output data files will be saved
path_processed_data = os.path.join(path_dataset, 'eis-processed')
os.makedirs(path_processed_data, exist_ok=True)


# Preprocessing via Lin-KK validation

In [30]:
for fpath in tqdm(file_list, desc="Processing EIS data"):
    df = pd.read_csv(fpath, sep="\t")
    if df.empty:
        print(f"File {fpath} is empty")
        continue

    # ?: Rename the columns
    # new_col = [
    #     "freq/Hz", "Re(Z)/Ohm", "=-@Im(Z)/@Ohm", "|Z|/Ohm", "Phase(Z)/deg", "time/s",
    #     "<Ewe>/V", "<I>/mA", "Cs/µF", "Cp/µF", "cycle number", "I Range", "|Ewe|/V",
    #     "|I|/A", "Ns", "(Q-Qo)/mA.h", "Re(Y)/Ohm-1", "Im(Y)/Ohm-1", "|Y|/Ohm-1",
    #     "Phase(Y)/deg", "dq/mA.h",
    # ]
    # df.columns = new_col

    freq = np.array(df["freq/Hz"]).astype(float)
    reals = np.array(df["Re(Z)/Ohm"]).astype(float)
    imags = -np.array(df["-Im(Z)/Ohm"]).astype(float)
    impedance = reals + imags * 1j

    # Preprocess the impedance data
    try:
        eis_data, R, rmse = ae.preprocess_impedance_data(impedance, freq, threshold=0.05)
    except Exception as e:
        print(f"Cannot preprocess due to {e}")
        continue

    # Save preprocessed data to csv
    saveto = os.path.join(path_processed_data, os.path.basename(fpath))
    saveto = saveto.replace('.txt', '.csv')
    eis_data.to_csv(saveto, header=None, index=False)


Processing EIS data:   0%|          | 0/256 [00:00<?, ?it/s]

File dataset/JonesLee2022/raw-data/fixed-discharge/PJ131/PJ131_033_15_GEIS_CA3.txt is empty


# Sort data into charge/dischage conditions

In [31]:
path_sorted_data = os.path.join(path_dataset, 'eis-sorted')

# Create subfolders to store charge/discharge files
path_discharge_data = os.path.join(path_sorted_data, 'discharge')
path_charge_data = os.path.join(path_sorted_data, 'charge')

os.makedirs(path_discharge_data, exist_ok=True)
os.makedirs(path_charge_data, exist_ok=True)

file_list = glob.glob(os.path.join(path_processed_data, '*.csv'))

# Loop through the processed files and copy them to charge/discharge folders
for fpath in file_list:
    test_condition = get_test_condition(fpath)
    if test_condition == "discharge":
        shutil.copy(fpath, path_discharge_data)
    elif test_condition == "charge":
        shutil.copy(fpath, path_charge_data)
    else:
        print(f"Cannot sort {fpath}")
